<font color='black'>
<h1>
<span style="font-family:verdana; font-size:1.4em;">
Handling Corrupt Records
</span>
</h1>
</font>

<font color='steelblue'>
<h2>
<span style="font-family:verdana; font-size:1.4em;">
Example of handling corrupt records in the dataset<br>
</span>
</h2>
</font>

<font color='grey'>
<span style="font-family:verdana; font-size:1.2em;">
    <b>Following is included in this notebook</b>:
    <ul>
        <li>Understand and use different modes of handling corrupt records</li>
        <li><b>Databricks only</b>, Use badRecordsPath option</li>
    </ul>
</span>
</font>

<font color='tomato'>
<span style="font-family:verdana; font-size:1.2em;">
    <b>IF RUNNING IN DATABRICK environment, set the next cell to True</b>:
</span>
</font>

In [ ]:
DATA_BRICKS = False

In [ ]:
if DATA_BRICKS == False:
    # Set up the environment for using pyspark
    import findspark
    findspark.init()
    
    from pyspark.sql import SparkSession
    from pyspark import SparkConf, SparkContext
    
    spark = SparkSession.builder.appName("Corrupt Records").getOrCreate()
    sc = spark.sparkContext
    sc.setLogLevel("Error")

In [ ]:
# Data type definitions, # Lot of functions used in this notebook
from pyspark.sql.types import *
from pyspark.sql.functions import *

## Utility Functions

In [ ]:
def showDataFrame(df, numrecs = 5):
    if DATA_BRICKS == False:
        df.limit(numrecs).show(truncate = False)
    else:
        display(df)

<font color='gray'>
<h3>
<span style="font-family:verdana; font-size:1.2em;">
Handling Corrupt Data<br>
</span>
</h3>
</font>
<span style="font-family:verdana; font-size:1.0em;">
When handling ETL pipeline, need efficient methods to handle corrupt data. As the data size and complexitity of data increase so does the data corruption.<br>
When building a pipleline, want to ensure that the pipeline behaves as expected.<br>
Data corruption includes:
    <ul>
        <li>Missing Data</li>
        <li>Schema errors (mismatch)</li>
        <li>Incomplete information</li>
        <li>Data types issues (different datatypes within a column)</li>
    </ul>
    <b>Options for handling corrupt records:</b>
</span>

| ParseMode | Behavior |
|-------------|----------|
| **`PERMISSIVE`** | creates a "_corrupt_record" column for corrupt records (by default) |
| **`DROPMALFORMED`** | ignores/drops all corrupted records |
| **`FAILFAST`** | throws an exception when it meets corrupted records - usually used in testing a pipeline|

### Here is a simple example of illustrating the 3 options discussed above

### PERMISSIVE mode

In [ ]:
data = """{"x": 1, "y":2, "z":3}|{"x": 1, "y":2, "z":3}|{"x": 1, "y, "z":10}""".split('|')

In [ ]:
# Create a column for corrupt records
corruptDF = (spark.read
  .option("mode", "PERMISSIVE")
  .option("columnNameOfCorruptRecord", "_corrupt_record")
  .json(sc.parallelize(data))
)

In [ ]:
showDataFrame(corruptDF)

### DROPMALFORMED mode

In [ ]:
# Drop Corrupt records
corruptDF = (spark.read
  .option("mode", "DROPMALFORMED")
  .json(sc.parallelize(data))
)

In [ ]:
showDataFrame(corruptDF)

### FAILFAST mode<br>
<span style="font-family:verdana; font-size:1.0em;">
Throws an exception to stop the processing in the ETL job
</span>

In [ ]:
# throw exception and stop processing during testing of ETL job
# this code will throw exception
try:
    corruptDF = (spark.read
                 .option("mode", "FAILFAST")
                 .json(sc.parallelize(data))
                )
    showDataFrame(corruptDF)

except Exception as e:
    print("THROWING EXCEPTION")
    print(e)